# Push the model's prediction to Kaggle

### Load the model and the data


In [3]:
import pickle
import numpy as np
import pandas as pd
from lightning.pytorch import Trainer
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from dmsensei.core import DataModule

# model = pickle.load(open('model.pkl','rb'))
trainer = Trainer()

dm = DataModule(
            name=["ribonanza"],
            force_download=False,
            batch_size=256,
            num_workers=1,
            train_split=0,
            valid_split=0,
            predict_split=1.,
            overfit_mode=False,)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/yvesmartin/.pyenv/versions/3.11.6/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


Using local data for: ribonanza
Using local data for: CT_files_pdbee
Using local data for: sarah_supermodel
Using local data for: utr
Using local data for: SARS2
Using local data for: pri-miRNA


In [13]:
# rebuild this into a series of lines

REFERENCE_METRIC = {
    'structure': 'f1', 
    'dms': 'mae',
    'shape': 'mae'
}

from dmsensei.core.metrics import metric_factory
from torch import tensor

lines = []
for a in dm.predict_dataloader():
    break
data, metadata = a

for idx in range(len(metadata['index'])):
    line = {}
    for k, v in metadata.items():
        line[k] = v[idx]
    # for k, v in predictions.items():
    #     line['pred_{}'.format(k)] = v[idx]
    lines.append(line)
for data_type, vals in data.items():
    for k,v in zip(vals['index'], vals['values'].tolist()):
        name = "true_{}".format(data_type) if data_type != 'sequence' else 'sequence'
        lines[k.item()][name] = v
        lines[k.item()][name.replace('true', 'pred')] = v
        
for data_type in ['dms','shape']:
    for line in lines:
        if not ('true_{}'.format(data_type) in line and 'pred_{}'.format(data_type) in line):
            continue
        line['score_{}'.format(data_type)] = metric_factory[REFERENCE_METRIC[data_type]](pred=tensor(line['pred_{}'.format(data_type)]), true=tensor(line['true_{}'.format(data_type)]), batch=False)
    
        


In [14]:
pd.DataFrame(lines)

,length,reference,index,quality,true_shape,pred_shape,sequence,score_shape,true_dms,pred_dms,score_dms
0,177,738c2162fa4b,0,1.0,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...","[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...","[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, ...",0.0,NaN,NaN,NaN
1,177,1157e0faa2cc,1,1.0,NaN,NaN,"[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, ...",NaN,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...","[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...",0.0
2,177,111f064a5ee6,2,1.0,NaN,NaN,"[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, ...",NaN,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...","[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...",0.0
3,177,d2480f45adc5,3,1.0,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...","[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...","[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, ...",0.0,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...","[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...",0.0
4,170,49e34c607caa,4,1.0,NaN,NaN,"[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, ...",NaN,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...","[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...",0.0
...,...,...,...,...,...,...,...,...,...,...,...
251,115,901ffc55a456,251,1.0,NaN,NaN,"[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, ...",NaN,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...","[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...",0.0
252,177,da88695a1719,252,1.0,NaN,NaN,"[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, ...",NaN,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...","[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...",0.0
253,177,2eba20937299,253,1.0,NaN,NaN,"[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, ...",NaN,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...","[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...",0.0
254,177,dc27b082629b,254,1.0,NaN,NaN,"[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, ...",NaN,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...","[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...",0.0


In [8]:
data, metadata = a
# rebuild this into a series of lines
df_lines = pd.DataFrame(metadata)
for data_type, arr in data.items():
    df_lines = df_lines.merge(
        pd.DataFrame(
             pd.Series(arr['values'].tolist(), index=arr['index'].tolist()), columns=[data_type]),
         how='outer', left_index=True, right_index=True)
df_lines    

,length,reference,index,quality,dms,shape,sequence
0,177,738c2162fa4b,0,1.0,NaN,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...","[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, ..."
1,177,1157e0faa2cc,1,1.0,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...",NaN,"[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, ..."
2,177,111f064a5ee6,2,1.0,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...",NaN,"[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, ..."
3,177,d2480f45adc5,3,1.0,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...","[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...","[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, ..."
4,170,49e34c607caa,4,1.0,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...",NaN,"[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, ..."
...,...,...,...,...,...,...,...
251,115,901ffc55a456,251,1.0,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...",NaN,"[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, ..."
252,177,da88695a1719,252,1.0,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...",NaN,"[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, ..."
253,177,2eba20937299,253,1.0,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...",NaN,"[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, ..."
254,177,dc27b082629b,254,1.0,"[-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...",NaN,"[3, 3, 3, 1, 1, 2, 3, 1, 2, 4, 2, 3, 1, 3, 4, ..."


In [26]:


import json

json.dump(lines, open('t.json', 'w'))

In [5]:
metadata

,length,reference,index,quality
0,177,738c2162fa4b,0,1.0
1,177,1157e0faa2cc,1,1.0
2,177,111f064a5ee6,2,1.0
3,177,d2480f45adc5,3,1.0
4,170,49e34c607caa,4,1.0
...,...,...,...,...
251,115,901ffc55a456,251,1.0
252,177,da88695a1719,252,1.0
253,177,2eba20937299,253,1.0
254,177,dc27b082629b,254,1.0


In [20]:
metadata = pd.DataFrame(a[1])
metadata

,length,reference,index,quality
0,177,738c2162fa4b,0,1.0
1,177,1157e0faa2cc,1,1.0
2,177,111f064a5ee6,2,1.0
3,177,d2480f45adc5,3,1.0
4,170,49e34c607caa,4,1.0
...,...,...,...,...
251,115,901ffc55a456,251,1.0
252,177,da88695a1719,252,1.0
253,177,2eba20937299,253,1.0
254,177,dc27b082629b,254,1.0


In [42]:
a[0]['dms']['values'].tolist()[0]

[-1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 -1000.0,
 0.22300000488758087,
 0.05999999865889549,
 0.023000000044703484,
 1.0,
 1.0,
 -1000.0,
 1.0,
 0.4650000035762787,
 0.3619999885559082,
 0.6990000009536743,
 0.4970000088214874,
 -1000.0,
 -1000.0,
 0.39899998903274536,
 0.25,
 0.8700000047683716,
 0.5839999914169312,
 0.32199999690055847,
 0.2639999985694885,
 -1000.0,
 0.382999986410141,
 -1000.0,
 -1000.0,
 0.6779999732971191,
 0.5299999713897705,
 1.0,
 -1000.0,
 0.10199999809265137,
 0.16899999976158142,
 0.16899999976158142,
 0.44999998807907104,
 -1000.0,
 -1000.0,
 0.7879999876022339,
 -1000.0,
 -1000.0,
 0.3160000145435333,
 0.25999999046325684,
 0.40700000524520874,
 -1000.0,
 -1000.0,
 -1000.0,
 0.5849999785423279,
 0.7450000047683716,
 1.0,
 -1000.0,
 -1000.

In [45]:
pd.Series(a[0]['dms']['values'].tolist(), index=a[0]['dms']['index'].tolist())

1      [-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...
2      [-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...
3      [-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...
4      [-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...
5      [-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...
                             ...                        
251    [-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...
252    [-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...
253    [-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...
254    [-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...
255    [-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...
Length: 245, dtype: object

### Run the prediction

In [ ]:
prediction = trainer.predict(
        model,
        datamodule=DataModule(
            name=["ribo-test"],
            data='sequence',
            force_download=False,
            batch_size=256,
            num_workers=1,
            train_split=0,
            valid_split=0,
            predict_split=1.,
            overfit_mode=False,
        ),
    )

### Format the prediction and save it locally

In [ ]:
import json

def format_to_ribonanza(prediction):
    
    # load data (EDIT THIS)
    data = json.load(open('/root/DMSensei/scripts/data/input_files/ribo-test/data.json'))
    
    # reformat into individual sequences
    arr = [p for batch in prediction for p in batch]
    
    # remove padding
    arr = [a[:s] for a,s in zip(arr, [len(d['sequence']) for d in data.values()])] 
    
    # stack into dataframe
    prediction = np.vstack(arr)
    return pd.DataFrame(prediction, columns=["reactivity_DMS_MaP", "reactivity_2A3_MaP"]).reset_index().rename(columns={"index": "id"})

format_to_ribonanza(prediction).to_csv('submission.csv', index=False)

### Push the prediction to Kaggle

Note: setup your Kaggle authentification first:
1. download your Kaggle API keys `kaggle.json` here: https://www.kaggle.com/settings/account
2. save it to `~/.kaggle/kaggle.json`:

    ```bash
    mv ~/Downloads/kaggle.json ~/.kaggle/kaggle.json
    ```
3. push your results to kaggle using the following line

In [ ]:
os.system('kaggle competitions submit -c stanford-ribonanza-rna-folding -f submission.csv -m "test commit"')